In [ ]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg as scylin
from time import time
import os
from datetime import datetime

from Control.PSO import PSO

omega0 = 1.0
gamma = [0.05]
T = 5.0
tnum = 1250
tspan = np.linspace(0, T, tnum)
dt = tspan[1]-tspan[0]
cnum = tnum
vx = 0.5*np.ones(cnum)
vy = 0.5*np.ones(cnum)
vz = 0.5*np.ones(cnum)

sx = np.array([[0.+0.j, 1.+0.j],[1.+0.j, 0.+0.j]])  
sy = np.array([[0.+0.j, 0.-1.j],[0.+1.j, 0.+0.j]]) 
sz = np.array([[1.+0.j, 0.+0.j],[0.+0.j, -1.+0.j]])
sp, sm = 0.5*(sx+1.j*sy), 0.5*(sx-1.j*sy)

#initial state
psi0 = np.array([[1.+0.j],[0.+0.j]])
psi1 = np.array([[0.+0.j],[1.+0.j]])
psi_p = (psi0+psi1)/np.sqrt(2)
psi_m = (psi0-psi1)/np.sqrt(2)
rho0 = np.dot(psi_p, psi_p.conj().T)
dim = len(rho0)

#time independent Hamiltonian
H0 = 0.5*omega0*sz
dH0 = [0.5*sz]

#control Hamiltonian
Hc_ctrl = [sx,sy,sz]
Hc_coeff = [vx,vy,vz]


#measurement
M1 = np.dot(psi_p, psi_p.conj().transpose())
M2 = np.dot(psi_m, psi_m.conj().transpose())
M  = [M1, M2]

Lvec = [sz]
particle_num = 10
PSO = PSO(particle_num, tspan, rho0, H0, Hc_ctrl, dH0, Hc_coeff, Lvec, gamma, control_option=True, c0=1.0, c1=2.0, c2=2.0, seed=200)

#==========================================================
dayTime = datetime.now().date().strftime('%Y%m%d')
path = str(dayTime)+'test_GRAPE'

isexists=os.path.exists(path)
if not isexists:
    os.makedirs(path)
if os.path.exists('./'+path+'/'+'ctrl.txt'):
    os.remove('./'+path+'/'+'ctrl.txt')
if os.path.exists('./'+path+'/'+'F.txt'):
    os.remove('./'+path+'/'+'F.txt')
#==========================================================

save_num = 1
episode = 100
PSO.swarm_origin()  
for round_i in range(episode):
    PSO.iteration()
    if round_i%save_num == 0 or round_i == episode-1:
        fvx = open('./'+path+'/'+'ctrl.txt','a')
        fvx.write('\n')
        np.savetxt(fvx, np.array(PSO.gbest).T)
        fvx.close()
        
        f = open('./'+path+'/'+'F.txt','a')
        f.write('\n')
        np.savetxt(f, np.array([PSO.F]).T)
        f.close()
        print(round_i,PSO.F)
        

In [ ]:
plt.plot(np.linspace(0,max_iter, max_iter), fitness, 'r-')
plt.xlabel('episode',fontsize=20)
plt.xlabel('$F$',fontsize=20)
plt.axis([0, 5., 0, 10])
plt.xticks([0,1,2,3,4,5],fontsize=15)
plt.yticks([0,5,10],fontsize=15)
plt.legend(bbox_to_anchor = [0.8, 1.0], frameon=False,fontsize=15)

plt.show() 